# A first look at data

date: 12/04/2018

Example code for reading and checking data


In [1]:
import pandas as pd
import dask.dataframe as dd
import glob as glob
import numpy as np
from scipy.stats import kde

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib.colors import LogNorm
plt.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

In [ ]:
filename = '/data/IceCube/2015/output_126836_IC86.2015.hdf'
dataGen = pd.read_hdf(filename, key='Laputop', columns=['x'])
dataGen.drop(dataGen.columns.difference(['x']), 1, inplace=True)
print(dataGen)

In [ ]:
def merge_and_select_data(filenames, output, number_of_files=10000):
    store =pd.HDFStore('/data/IceCube/merged2015.h5',complib='blosc')
    for i, file in enumerate(filenames):
        print(i, file)
        s = pd.HDFStore(file)
        dataGen = s.select('Laputop')
        dataGen.drop(dataGen.columns.difference(['Run', 'Event','x','y','z','zenith','azimuth','time']), 1, inplace=True)
        dataRec = s.select('LaputopParams')
        dataRec.drop(dataRec.columns.difference(['s125','beta','chi2','ndf']), 1, inplace=True)
        dataCuts = s.select('IT73AnalysisIceTopQualityCuts')
        dataCuts.drop(['Run', 'Event','SubEvent','SubEventStream'], 1, inplace=True)
        dataGen = dataGen.dropna(axis=1, how='all')
        dataRec = dataRec.dropna(axis=1, how='all')
        data = pd.concat([dataGen, dataRec, dataCuts], axis=1, join_axes=[dataGen.index])
    #print(data)
        data = data.query('exists!=0 & IceTopMaxSignalInside!=0 & IceTop_reco_succeeded!=0'
                         ' & IceTop_StandardFilter!=0 & IceTopMaxSignalInside !=0 '
                         ' & Laputop_FractionContainment!=0 & BetaCutPassed!=0 &s125>1')
        data.drop(data.columns.difference(['Run','Event','x','y','z','zenith','azimuth',
                                          's125','beta','chi2','ndf']), axis=1,inplace=True)
    
        store.append(key ='data', value=data,  format='t',chunksize=200000)
        s.close()
        if i>number_of_files:
            break

    return store

In [2]:

filenames = ['/data/IceCube/2015/output_126836_IC86.2015.hdf',
             '/data/IceCube/2015/output_126938_IC86.2015.hdf']
filenames = glob.glob('/data/IceCube/2015/*')
dataAll = pd.DataFrame() 
store =pd.HDFStore(,complib='blosc')
number_of_files = 10000
store = merge_and_select_data(filenames, '/data/IceCube/merged2015.h5', number )        
dataAll = store.select('data')
print(dataAll.count())
store.close()    




0 /data/IceCube/2015/output_126468_IC86.2015.hdf
1 /data/IceCube/2015/output_126417_IC86.2015.hdf
2 /data/IceCube/2015/output_126561_IC86.2015.hdf
3 /data/IceCube/2015/output_126409_IC86.2015.hdf
4 /data/IceCube/2015/output_126455_IC86.2015.hdf
5 /data/IceCube/2015/output_127228_IC86.2015.hdf
6 /data/IceCube/2015/output_126396_IC86.2015.hdf
7 /data/IceCube/2015/output_126496_IC86.2015.hdf
8 /data/IceCube/2015/output_126855_IC86.2015.hdf
9 /data/IceCube/2015/output_126503_IC86.2015.hdf
10 /data/IceCube/2015/output_126735_IC86.2015.hdf
11 /data/IceCube/2015/output_126847_IC86.2015.hdf
12 /data/IceCube/2015/output_127256_IC86.2015.hdf
13 /data/IceCube/2015/output_126474_IC86.2015.hdf
14 /data/IceCube/2015/output_127301_IC86.2015.hdf
15 /data/IceCube/2015/output_126679_IC86.2015.hdf
16 /data/IceCube/2015/output_126787_IC86.2015.hdf
17 /data/IceCube/2015/output_126605_IC86.2015.hdf
18 /data/IceCube/2015/output_126816_IC86.2015.hdf
19 /data/IceCube/2015/output_126690_IC86.2015.hdf
20 /data/I

In [ ]:
#print(dataCuts.head())
#print(dataGen.head())
#print(dataRec.head())


In [ ]:
hist = plt.hist(np.log10(dataAll.s125), bins=150)
#plt.hist(np.log10(df.s125), 'ro')